In [12]:
# !pip install datasets
# !huggingface-cli login

In [13]:
# from datasets import load_dataset
# load_dataset("balochiml/balochi-language-data", data_dir="data", cache_dir="../data")

# Generate the processed data without English characters

In [14]:
import os

def get_txt_file_paths(directory):
    txt_file_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                txt_file_paths.append(file_path)
    return txt_file_paths

# Replace "directory_path" with the actual path of the directory you want to search
directory_path = "../data/raw_text"
txt_paths = get_txt_file_paths(directory_path)

len(txt_paths)


4294

In [15]:
import re

def clean_text(file_path):
    # Open the file and read it into memory
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    # Remove English-language characters and numbers
    text = re.sub(r'[a-zA-Z0-9]', '', text)

    # Remove any excess whitespace
    text = re.sub(r'[^\S\n]+', ' ', text)

    return text

In [16]:
for path in txt_paths:
    cleaned_text = clean_text(path)

    # write the cleaned text to a new file with an incremented filename
    # write the files all into the '../data/processed_text' directory
    with open(f'../data/processed_text/{path.split("/")[-1]}', 'w', encoding='utf-8') as file:
        file.write(cleaned_text)


In [17]:
from tokenizers import Tokenizer
from tokenizers.models import BPE

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

In [18]:
from tokenizers.pre_tokenizers import Whitespace
tokenizer.pre_tokenizer = Whitespace()

In [52]:
from tokenizers.trainers import BpeTrainer

trainer = BpeTrainer(
    min_frequency=2,
    vocab_size=30000,
    special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
    show_progress=True,
)

In [53]:
# get a list of all the txt files in
# '/Users/strickvl/balochi/balochi-tokenizer/data/processed_text'

processed_files = get_txt_file_paths("../data/processed_text")
assert len(processed_files) == len(txt_paths)
len(processed_files)

4294

In [54]:
tokenizer.train(processed_files, trainer)

In [55]:
tokenizer.model

In [56]:
tokenizer.get_vocab_size()

30000

In [57]:
# tokenizer.get_vocab()

In [58]:
tokenizer.save("../models/30k-balochi-tokenizer.json")

In [59]:
tokenizer = Tokenizer.from_file("../models/30k-balochi-tokenizer.json")

In [62]:
sample_text = "      آیک  جناورے اَت۔  لھتے گشیت آ سکیں کارزوالے ات کہ اگاں آزاتی دیگ بہ بیت، بازارءَ، لوگے ءَ، جاگاہ یے  ءَ،دپتر ء ُ کارگس یے  ءَ یا ھر ھما جاگاہ ءَ کہ شُت کنت مزنیں کارزوالی کنت۔گوں ھر کس ءَ جنگ ء ُ مڑ بیت۔گدء ُ پچاں  چنڈ چنڈ ء ُ راڑ راڑ کنت،کاگد ء ُ وانگیاں وارت ء ُ آدراہ کنت۔ورگی چیزاں اگاں وارت نکنت آھاں گٹ پاچیت ھراب کنت۔ایندگہ جناور چہ بندات ء َ ایشی ءِ کازوالیاں چہ وتا دیر دارگ ءِ کوشست کن اَنت۔ چیا کہ آ بازیں دگہ ھرابی ء ُ کارزوالی ھم کنت،پمیشکا کسانیں جناور  بالی مُرگ،کوہ پاچن،آسک ء ُ ایندگہ کسان کسانیں جناورچر آئی ءِ کارزوالیانی سوب ءَ آئی ءَ چہ سک باز شزار اَنت ۔".replace("\xa0", "")
sample_sentence = sample_text.split("۔")[2]
sample_sentence

'گوں ھر کس ءَ جنگ ء ُ مڑ بیت'

In [63]:
tokenizer.encode(sample_sentence).tokens

['گوں', 'ھر', 'کس', 'ءَ', 'جنگ', 'ء', 'ُ', 'مڑ', 'بیت']